In [1]:
import os
# ensure the backend is set
import argparse
import torch
import keras
from torch.utils.data import DataLoader

In [2]:
os.environ["KERAS_BACKEND"] = "torch"

import bayesflow as bf
from dl_src.load_data import data_loader
from keras.src.backend.common import global_state

In [3]:
%load_ext autoreload
%autoreload 2

In [14]:
train, validation, inference_variables = data_loader()

In [11]:

summary_net = bf.networks.TimeSeriesTransformer(summary_dim=32,
																								time_axis=-1,
																								time_embedding="time2vec")

inference_net = bf.networks.FlowMatching(
	integrator = "rk2",
		subnet_kwargs={"residual": True, "dropout": 0.0, "widths": (128, 128, 128, 128)}
)

# workflow = bf.BasicWorkflow(
# 		adapter=bf.adapters.Adapter(),
# 		inference_network=inference_net,
# 		summary_network=summary_net,
# 		inference_variables=train.inference_variables
# )

# get num_batches from train_loader
initial_learning_rate = 1e-3
epochs = 10
		# Create optimizer
scheduled_lr = keras.optimizers.schedules.CosineDecay(
		initial_learning_rate=initial_learning_rate,
		decay_steps=epochs*train.num_batches,
		alpha=1e-8
)
optimizer = keras.optimizers.Adam(learning_rate=scheduled_lr)


# Create approximator
approximator = bf.ContinuousApproximator(
		summary_network=summary_net,
		inference_network=inference_net,
		adapter=None
)
approximator.compile(optimizer=optimizer)


In [ ]:
# Train and compute the average of last 5 validation losses
history = approximator.fit(
    epochs=epochs,
    dataset=train,
    validation_data=validation
)